<h1>Final assignment: Voorspellen met Monte Carlo</h1>
<h2>Door: Dennis Besselsen V2A</h2>


In dit notebook leg ik uit hoe ik met gebruik van verschillende Pseudo number generators, een voorbeeld voetbal toernooi kan simuleren monte-carlo style.

Als eerst ga ik de tabel met kansen (winst/ gelijk/ verlies tijdens thuiswedstrijden) vullen aan de hand van de data op canvas.

Dit doe ik d.m.v. van een dictionary.

Elke team heeft 1 key in de dict. Elke value is een lijst van kansen. De order van values in deze lijsten is gelijk als die van de keys, ofwel:

 - index[0]=Ajax
 - index[1]=Feyenoord
 - index[2]=PSV
 - index[3]=FC Utrecht
 - index[4]=Willem II

Teams kunnen niet tegen zichzelf spelen, dus deze plekken in de lijsten worden gevuld met None.

In [62]:
chance_dict = { 'Ajax':         [None,      '65/17/18', '54/21/25', '74/14/12', '78/13/9'   ],
                'Feyenoord':    ['30/21/49',None,       '37/24/39', '51/22/27', '60/21/19'  ],
                'PSV':          ['39/22/39','54/22/24', None,       '62/20/18', '62/22/16'  ],
                'FC Utrecht':   ['25/14/61','37/23/40', '29/24/47', None,       '52/23/25'  ],
                'Willem II':    ['17/18/65','20/26/54', '23/24/53', '37/25/38', None        ]}
                

<h1>De eerste PRNG : Middle Square PRNG</h1>
SOURCE: Canvas /  tijdens de les doorgenomen

Deze PRNG zal outputs repeaten na ongeveer 90 tries, afhankelijk van de exacte seed. Om deze reden is de Middle Square PRNG niet geschikt voor het simuleren van dit toernooi, maar het leek me toch leuk om hem mee te nemen.


In [63]:
class MidSquareGen(object):   # Algemene opzet voor een RNG object: init, en next. Optioneel (re)set seed.
    def __init__(self, seed):
        self.seed = seed
        
    def reset(self, seed):
        self.seed = seed
        
    def next(self):
        self.seed = MidSquareGen.select_middle(self.seed**2, 4) # Bereken volgende seed (en bewaar die)
        return self.seed / 10000.0                 # Bereken waarde uit seed (kan de geschaalde seed zijn, of een complexere functie)

    @staticmethod
    def select_middle(number, positions):       # Statische helperfunctie
        left = True
        while len(str(number)) > positions:
            if left: 
                number = int(str(number)[1:])
                left = False
            else:
                number = int(str(number)[:-1])
                left = True
        return number
        
# start seed door willekeurig op mijn toetsenbord te rammen :)
rng_ms = MidSquareGen(5566778885464564563567324524978654423)

<h1>De eerste PRNG : Mersenne Twister PRNG</h1>
Sources: https://github.com/yinengy/Mersenne-Twister-in-Python       &        https://en.wikipedia.org/wiki/Mersenne_Twister

Deze PRNG zal outputs repeaten na ongeveer 19000 tries, afhankelijk van de exacte seed. Dit PRNG is daarom veel beter geschikt voor het simuleren van dit toernooi.


In [72]:

class Twister():
    def __init__(self, c_seed=0):
        # standaard MT19937 die een 32-bit word length gebruikt.
        # w is de word lengte in bits.
        # n is de mate van herhaling.
        # m is de offset van het middelste word
        # r is het scheidings punt van woorden
        (self.w, self.n, self.m, self.r) = (32, 624, 397, 31)
        # a is de coëfficiënt van de rationale normaalvorm 
        self.a = 0x9908B0DF
        
        # s & t zijn hier bitshifts
        # b & c zijn bitmasks
        (self.s, self.b) = (7, 0x9D2C5680)
        (self.t, self.c) = (15, 0xEFC60000)
         # u & l zijn extra bitshifts en d is een extra bitmasks
        (self.u, self.d, self.l) = (11, 0xFFFFFFFF, 18)
        
        self.f = 1812433253
        # make a arry to store the state of the generator
        self.MT = [0 for i in range(self.n)]
        self.index = self.n+1
        self.lower_mask = 0x7FFFFFFF
        self.upper_mask = 0x80000000
        # inital the seed
        self.c_seed = c_seed
        self.seed(c_seed)

    def seed(self, num):
        """initialize the generator from a seed"""
        self.MT[0] = num
        self.index = self.n
        for i in range(1, self.n):
            temp = self.f * (self.MT[i-1] ^ (self.MT[i-1] >> (self.w-2))) + i
            self.MT[i] = temp & 0xffffffff

    def twist(self):
        """ Generate the next n values from the series x_i"""
        for i in range(0, self.n):
            x = (self.MT[i] & self.upper_mask) + \
                (self.MT[(i+1) % self.n] & self.lower_mask)
            xA = x >> 1
            if (x % 2) != 0:
                xA = xA ^ self.a
            self.MT[i] = self.MT[(i + self.m) % self.n] ^ xA
        self.index = 0

    def extract_number(self):
        """ Extract a tempered value based on MT[index]
            calling twist() every n numbers
        """
        if self.index >= self.n:
            self.twist()

        y = self.MT[self.index]
        y = y ^ ((y >> self.u) & self.d)
        y = y ^ ((y << self.s) & self.b)
        y = y ^ ((y << self.t) & self.c)
        y = y ^ (y >> self.l)

        self.index += 1
        return y & 0xffffffff

    def random(self):
        """ return uniform ditribution in [0,1) """
        return self.extract_number() / 2**self.w  

#Seed '1337420' heb ik handmatig ingevuld 
rng_mt = Twister(1337420)


<h1>Het simuleren van het toernooi</h1>
Ik wil de wedstrijden als volgt simuleren

Voorbeeld: Team A(home) tegen Team B(away) heeft als kansen: 50/40/10. ( 50% kans om te winnen, 40% op gelijk spel, 10% kans op verlies)

 Deze waardes zet ik om naar: 0.50/0.40/0.10

Vervolgens laat ik een van de boven getoonde PRNG een getal tussen te 0 en 1 genereren.

Als dit Pseudo-random getal onder 0.5 valt of exact 0.5 is, heeft team A gewonnen.  

Als dit Pseudo-random getal boven 0.5 is maar onder (0.5 + 0.4), hebben de teams gelijk gespeeld.

Alle overige opties duiden op het verlies van team A.





In [68]:

def WDL_sim(chance_dict,generator):
    
    # Dictionary waar de score wordt bijgehouden
    score_dict = { 'Ajax':0,
                'Feyenoord':0,
                'PSV':0,
                'FC Utrecht':0,
                'Willem II':0}

    # Dictionary wat gebruikt wordt voor het verdelen van punten aan de 'away' teams
    away_dict = { 0:'Ajax',
                1:'Feyenoord',
                2:'PSV',
                3:'FC Utrecht',
                4:'Willem II'}


    for home_team,chances in chance_dict.items():
        for chance in chances:
            if chance != None:

                # split de value op elke  '/' , en deel ze waardes door 100 zodat het komma getallen worden tussen 0 en 1
                float_list = [(float(s)/100) for s in chance.split('/')]

                # win_perc is de eerste value in de list, loss_perc is de eerste 2 values in de list bij elkaar opgeteld.
                win_perc = float_list[0]
                loss_perc = float_list[1]+float_list[0]
                
                # Gebruik Mersenne Twister PRNG
                if generator == 'mt':
                    gen_chance = rng_mt.random()
                # Gebruik Middle Square PRNG    
                elif generator == 'ms':                   
                    gen_chance = rng_ms.next()
                else:
                    print("Er is geen geldige PRNG aangegeven. ABORTING")
                
                    
                if gen_chance <= win_perc:
                    score_dict[home_team] += 3                              # home team points +3

                elif gen_chance > win_perc and gen_chance < loss_perc:
                    score_dict[home_team] += 1                              # home team points +1
                    score_dict[away_dict[chances.index(chance)]]+=1         # away team points +1
                    
                    
                else:  # alle andere opties zijn geelimineerd dus hier kan else gebruikt worden
                    score_dict[away_dict[chances.index(chance)]]+=3   # away team points +3
    
    # Hier worden de teams op score gesorteerd.  Het result is een list van teams op volgorde van position in het kampioenschap. De scores worden niet meegenomen.
    sorted_positions = sorted(score_dict, key=score_dict.get, reverse=True)
                 
    return sorted_positions
                
                

In [66]:
def sim_spam(iterations, generator):
    # dict waar ieder team hun kansen op plaatsen in het kampioenschap wordt bijgehouden.
    # VOORBEELD:  'Ajax': [61.64, 23.72, 9.95, 3.69, 1.0]         betekent dat Ajax in 61.64% van de simulations als eerste in het kampioenschap eindigde, 23.72% van de simulations als tweede etc.
    placement_dict = {  'Ajax':         [0,0,0,0,0],
                        'Feyenoord':    [0,0,0,0,0],
                        'PSV':          [0,0,0,0,0],
                        'FC Utrecht':   [0,0,0,0,0],
                        'Willem II':    [0,0,0,0,0]}
                
    for _ in range(iterations):
        position_list = WDL_sim(chance_dict,generator)
        for idx, val in enumerate(position_list):
            placement_dict[val][idx] +=1
    for team,placements in placement_dict.items():
        templist=[]
        for place in placements:
            templist.append(round(place / iterations*100,2))
        placement_dict[team] = templist
    return placement_dict

<h1>Results</h1>
<h2>Mersenne Twister</h2>

In [67]:
#100000 itteraties die gebruik maken van de mersenne twister PRNG
position_chances = sim_spam(100000, 'mt')
position_chances


{'Ajax': [61.57, 23.81, 10.09, 3.62, 0.9],
 'Feyenoord': [11.37, 26.18, 31.73, 21.02, 9.7],
 'PSV': [22.62, 35.48, 25.21, 12.4, 4.28],
 'FC Utrecht': [3.48, 10.55, 21.96, 37.54, 26.46],
 'Willem II': [0.96, 3.97, 11.0, 25.42, 58.65]}

<h2>Middle Square</h2>

In [71]:
#100000 itteraties die gebruik maken van de middle square PRNG
position_chances = sim_spam(100000, 'ms')
position_chances


{'Ajax': [30.77, 46.15, 15.38, 7.69, 0.0],
 'Feyenoord': [23.08, 15.38, 15.38, 38.46, 7.69],
 'PSV': [23.08, 15.38, 30.77, 7.69, 23.08],
 'FC Utrecht': [15.38, 15.38, 23.08, 15.38, 30.77],
 'Willem II': [7.69, 7.69, 15.38, 30.77, 38.46]}

In de results is te zien dat de middle square wat vreemde uitkomsten heeft.

Ajax zou volgens de middle squared simulaties nooit op de 5e plaats in het tournament eindigen. Dit slaat natuurlijk nergens op. De kans is daadwerkelijk is wel aanwezig, ookal is deze misschien heel klein.  Dit is te zien in de reults van de Mersenne twister simulaties.